In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K

from PIL import Image

%matplotlib inline

In [ ]:
IM_WIDTH = 300 # Keep width = 300 for ResNet50
IM_HEIGHT = 225 # Keep height = 225 for ResNet50

BATCH_SIZE = 16

SEED = 76643

data_path = '/home/ubuntu/Data/Skin/bcc_others/'

global_mean = np.load(data_path + 'mean_image_bcc_others.npz')['image'].astype(np.uint8)
mean_image = Image.fromarray(global_mean).resize((IM_WIDTH, IM_HEIGHT))

fig = plt.figure()
plt.imshow(mean_image)
plt.title('BCC_OTHERS_MEAN_IMAGE')
plt.show()

global_mean = np.asarray(mean_image, dtype=np.float32)
np.random.seed(SEED)

In [ ]:
def preprocess_train(image):
    gauss = np.random.normal(0., 1.5, (IM_HEIGHT, IM_WIDTH, 3))
    return (image - global_mean) + gauss

def preprocess_image(image):
    return image - global_mean

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import compute_class_weight

train_img = ImageDataGenerator(preprocessing_function=preprocess_image, rescale=1./255)

img_gen = ImageDataGenerator(preprocessing_function=preprocess_image, rescale=1./255)

train_gen = train_img.flow_from_directory(data_path + 'train', 
                                        target_size=(IM_HEIGHT, IM_WIDTH), 
                                        class_mode='binary', 
                                        shuffle=True, 
                                        batch_size=BATCH_SIZE, 
                                        classes=['NON_BCC', 'BCC'], 
                                          seed=SEED
                                       )

print('Training Set Generator done')
print('----------------------------------------')
val_gen = img_gen.flow_from_directory(data_path + 'val', 
                                      target_size=(IM_HEIGHT, IM_WIDTH), 
                                      class_mode='binary', 
                                      shuffle=True, 
                                      batch_size=BATCH_SIZE, 
                                      classes=['NON_BCC', 'BCC'],
                                      seed=SEED
                                     )

print('Validation Set Generator done')
print('----------------------------------------')
test_gen = img_gen.flow_from_directory(data_path + 'test', 
                                       target_size=(IM_HEIGHT, IM_WIDTH), 
                                       class_mode='binary', 
                                       shuffle=False, 
                                       batch_size=BATCH_SIZE, 
                                       classes=['NON_BCC', 'BCC'],
                                       seed=SEED
                                      )
print('Test Set Generator done.')
print('----------------------------------------')
y_train = train_gen.classes

weights = compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {i: weights[i] for i in range(len(weights))}

print(class_weights)
print('----------------------------------------')
print(train_gen.class_indices)

In [ ]:
num_hidden_units = 512
num_hidden_units_1 = 64
num_hidden_units_2 = 64
model_name = 'inceptionv3'
train_type = 'full_train'
data = 'bcc_others'

filename = 'final_' + data + '_' + model_name + '_' + str(num_hidden_units) + '_' + train_type + '_2_hidden_' + '.h5'
print(filename)

In [ ]:
def f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return ( 2*(precision * recall) / (precision + recall + K.epsilon()) )

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return (true_positives / (predicted_positives + K.epsilon()))

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return (true_positives / (possible_positives + K.epsilon()))

In [ ]:
from keras.applications.inception_v3 import InceptionV3
# from keras.applications.resnet50 import ResNet50
# from keras.applications.vgg16 import VGG16
# import inception_v4
from keras import Model
from keras.models import load_model
from keras.layers import (Dense, Activation, Dropout, 
                          BatchNormalization, Flatten, GlobalAveragePooling2D)

# inc = InceptionV3(include_top=False, input_shape=(IM_HEIGHT, IM_WIDTH, 3), weights='imagenet')
inc = load_model('Finalized Hierarchical Models/df_vasc_others_inceptionv3_512_full_train.h5', 
                 custom_objects={'f1': f1, 'precision': precision, 'recall': recall})
# x = inc.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)
# x = Dense(num_hidden_units)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.2)(x)
# out = Dense(1, activation='sigmoid')(x)
for _ in range(5):
    inc.layers.pop()
x = inc.layers[-1].output
x = Dense(num_hidden_units_1, name='d1')(x)
x = BatchNormalization(name='bn1')(x)
x = Activation('relu', name='a1')(x)
x = Dropout(0.5, name='do1')(x)

x = Dense(num_hidden_units_1, name='d2')(x)
x = BatchNormalization(name='bn2')(x)
x = Activation('relu', name='a2')(x)
x = Dropout(0.2, name='do2')(x)

out = Dense(1, activation='sigmoid', name='out')(x)


model = Model(inputs=inc.input, outputs=out)

last_layer_weights = model.layers[-1].get_weights()
last_layer_weights[1] += 1.

model.layers[-1].set_weights(last_layer_weights)

print(model.layers[-1].get_weights()[1])

for layer in inc.layers:
    layer.trainable = False

In [ ]:
print(model.summary())

In [ ]:
from keras.optimizers import Adam

model.compile(loss='binary_crossentropy', optimizer=Adam(5e-4), metrics=['accuracy'])

_ = model.fit_generator(train_gen, 
                        steps_per_epoch=train_gen.samples//BATCH_SIZE + 1, 
                        validation_data=val_gen, 
                        validation_steps=val_gen.samples//BATCH_SIZE + 1, 
                        shuffle=True, 
                        epochs=1, 
                        class_weight=class_weights)

In [ ]:
from keras.optimizers import Adam, RMSprop

for layer in model.layers:
    layer.trainable = True

# model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-4, decay=0.0005), metrics=['accuracy', precision, recall, f1])
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=5e-5, decay=5e-4), metrics=['accuracy', precision, recall, f1])
print('Model compiled')

In [ ]:
# Recompiling model AFTER running for x epochs.
# model.compile(loss='binary_crossentropy', optimizer=Adam(lr=5e-6, decay=5e-4), metrics=['accuracy', precision, recall, f1])
print('Model re-compiled')
filename = filename[:-4] + '_2.h5'
print(filename)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(filename, 
                             save_best_only=True, mode='max',
                             monitor='val_f1', verbose=1)
reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.05, mode='min', patience=5, verbose=1)
stopper = EarlyStopping(monitor='val_loss', mode='min', patience=7, verbose=1)

h = model.fit_generator(train_gen, 
                        steps_per_epoch=train_gen.samples//BATCH_SIZE + 1, 
                        validation_data=val_gen, 
                        validation_steps=val_gen.samples//BATCH_SIZE + 1, 
                        shuffle=True, 
                        epochs=17,
                        class_weight=class_weights, 
                        callbacks=[checkpoint, stopper, reducer], initial_epoch=12)

fig = plt.figure()
plt.plot(h.history['loss'], 'r-')
plt.plot(h.history['val_loss'], 'b-')
plt.title('Loss plot')
plt.show()

fig = plt.figure()
plt.plot(h.history['f1'], 'r-')
plt.plot(h.history['val_f1'], 'b-')
plt.title('F1 plot')
plt.show()

fig = plt.figure()
plt.plot(h.history['precision'], 'r-')
plt.plot(h.history['val_precision'], 'b-')
plt.title('Precision plot')
plt.show()

fig = plt.figure()
plt.plot(h.history['recall'], 'r-')
plt.plot(h.history['val_recall'], 'b-')
plt.title('Recall plot')
plt.show()

In [ ]:
from keras.models import load_model

m = load_model('nv_non_nv_inception_holdout_test_hist.h5', 
               compile=True, 
               custom_objects={'f1':f1, 
                               'precision':precision, 
                               'recall':recall})
print('Best model loaded')

In [ ]:
y_pred = model.predict_generator(test_gen, verbose=1)

In [ ]:
y_true = test_gen.classes

y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = 0

In [ ]:
from sklearn.metrics import classification_report, roc_curve, auc
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_true=y_true.ravel(), y_pred=y_pred.ravel()))

print(classification_report(y_true=y_true.ravel(), y_pred=y_pred.ravel()))

fpr, tpr, threshold = roc_curve(y_true, y_pred)
model_auc = auc(fpr, tpr)

print('AUC: {}'.format(model_auc))

In [ ]:
'''
[[247   8]
 [  9  43]]
             precision    recall  f1-score   support

          0       0.96      0.97      0.97       255
          1       0.84      0.83      0.83        52

avg / total       0.94      0.94      0.94       307

AUC: 0.8977752639517345

-----------------------
[[244  11]
 [  4  48]]
             precision    recall  f1-score   support

          0       0.98      0.96      0.97       255
          1       0.81      0.92      0.86        52

avg / total       0.96      0.95      0.95       307

AUC: 0.9399698340874812
'''